In [1]:
# CELDA 1: imports and mqtt configuration

import time
import numpy as np
from datetime import datetime

from fmi_mlc import fmi_gym
from mqtt_publisher import MQTTPublisher
import paho.mqtt.client as mqtt
from config import parameter as dfmu

BROKER_HOST = "localhost"
BROKER_PORT = 1883

# fmu parameters
action_names = dfmu['action_names']
action_min   = dfmu['action_min']

# 1) Publisher creation for observations and actions
pub = MQTTPublisher(host=BROKER_HOST, port=BROKER_PORT)

# 2) Dictionary to save latest action
latest_actions = { name: None for name in action_names }

# 3) Client for subscribing to openhab commands
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] connected to broker {rc}. Susbscribing to topics…")
    for name in action_names:
        topic = f"openhab/actuator/{name}" #this is just for this example, adjust the topics as you configured them in openhab 
        client.subscribe(topic, qos=1)
        print(f"    • [SIM] Subscribed to {topic}")

def on_message(client, userdata, msg):
    try:
        value = float(msg.payload.decode())
    except:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] Payload no numérico: {msg.payload!r}")
        return
    action_key = msg.topic.split("/")[-1]
    latest_actions[action_key] = value
    #print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] Command → {action_key} = {value}") just for debug, not necessary

client.on_connect = on_connect
client.on_message = on_message

# 4) Coonect and start loop for mqtt command subscriber
client.connect(BROKER_HOST, BROKER_PORT)
client.loop_start()

# 5) FMU gym environment inizialization
env = fmi_gym(dfmu)
initial_obs = env.reset()
print("► [SIM] fmu initialized, first observation:", initial_obs)



C:\Users\andre\AppData\Local\Temp\ipykernel_24620\2348136589.py:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


[21:09:11] [SIM] Conectado al broker MQTT con código 0. Suscribiendo tópicos…
    • [SIM] Suscrito a openhab/actuator/Cool_SP_SE_1F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_NW_1F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_NE_1F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_SW_1F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_SW_2F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_SE_2F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_NW_2F_fmu
    • [SIM] Suscrito a openhab/actuator/Cool_SP_NE_2F_fmu
► [SIM] Entorno FMU inicializado. Observación inicial: [23.97591678 23.97591678 23.96347221 23.96347221 23.95350137 23.95350137
 23.93494958 23.93494958]
[21:09:39] [SIM] COMANDO → Cool_SP_NE_1F_fmu = 21.5
[21:09:40] [SIM] COMANDO → Cool_SP_NE_2F_fmu = 23.0
[21:09:41] [SIM] COMANDO → Cool_SP_NW_1F_fmu = 24.5
[21:09:43] [SIM] COMANDO → Cool_SP_NW_2F_fmu = 34.5
[21:09:44] [SIM] COMANDO → Cool_SP_SE_1F_fmu = 24.0
[21:09:45] [SIM] COMANDO → Cool_SP_SE_2F_fmu

In [2]:
# CELDA 2: Principal simulation loop (MQTT listener y publshing)

STEP_SIZE = dfmu['fmu_step_size']
T_FINAL   = dfmu['fmu_final_time']

# Mapping each name for each action
action_idx = { name: idx for idx, name in enumerate(action_names) }

t = 0.0
obs = initial_obs

while t < T_FINAL:

    # 2) Contructing action vector
    action_vector = np.zeros(len(action_names), dtype=float)
    for name in action_names:
        idx = action_idx[name]
        val = latest_actions.get(name)
        action_vector[idx] = val if (val is not None) else action_min[idx]

    # 3) Dictionary converion and publishing actions
    action_dict = { name: float(action_vector[action_idx[name]]) for name in action_names }
    pub.publish_actions(action_dict, timestamp=env.fmu_time)

    # 4) advance simulation
    obs, reward, done, info = env.step(action_vector)

    # 5)Obserbation publishing
    obs_dict = { name: float(obs[i]) for i, name in enumerate(dfmu['observation_names']) }
    pub.publish_observations(obs_dict, timestamp=env.fmu_time)

    # 6) Advancing sim time
    t += STEP_SIZE
    time.sleep(0.005)

    if done:
        break

print("► [SIM] Simulation completed a t =", t, "seconds.")


► [SIM] Simulación completada a t = 8640000.0 segundos.


In [5]:
# CELDA 4: Cerrar conexiones MQTT
# CELDA 4: Cerrar conexiones MQTT al finalizar

pub.close()
client.loop_stop()
client.disconnect()
print("► [SIM] Mqtt conections terminated. Simulation ended.")


► [SIM] Conexiones MQTT cerradas. Fin de la simulación.
